In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pydicom
import xarray as xr

In [ ]:
from pymedphys._data import download
from pymedphys._dicom import constants, coords, dose

import xdose

In [ ]:
dicom_dose_filepaths = {orient: download.get_file_within_data_zip(
    "dicomorient_doses_only.zip", f"RD.DICOMORIENT.Dose_{orient}.dcm"
) for orient in constants.IMAGE_ORIENTATION_MAP}

rtdoses_by_orient = {orient: pydicom.dcmread(dicom_dose_filepaths[orient]) for orient in constants.IMAGE_ORIENTATION_MAP}

In [ ]:
xdoses_by_orient = {orient: xdose.xdose_from_dataset(rtdoses_by_orient[orient], coord_system="SUPPORT", decimals_to_round_coords=2)  for orient in constants.IMAGE_ORIENTATION_MAP}

xdoses_by_orient["HFS"]

In [ ]:
# Need method="nearest" if exact coordinate isn't suppled to sel():
try:
    xdoses_by_orient["HFS"].sel(x=0)
except KeyError:
    print("KeyError")

In [ ]:
xdoses_by_orient["HFS"].sel(x=0, method="nearest")

In [ ]:
# Very easily plot dose slice, already labelled with colorbar and units
def plot_xdose_tcs_at_point(xdose, point, coord_system="SUPPORT"):
    LAT_2_LONG_RATIO = xdose.x.size / xdose.y.size   
    VERT_2_LONG_RATIO = xdose.z.size / xdose.y.size   

    fig, axes = plt.subplots(figsize=(12, 8), ncols=2, nrows=2, gridspec_kw={"width_ratios":(LAT_2_LONG_RATIO,1), "height_ratios":(VERT_2_LONG_RATIO,1)})
    axes[1, 1].axis('off')
    for ax in axes.ravel():
        ax.set_aspect("equal")

    try:
        xdose.sel(y=point[1], method="nearest").plot(ax=axes[0, 0], cmap="jet", vmin=0, vmax=xdose.max())
        axes[0, 0].axhline(y=point[2], color="silver")
        axes[0, 0].axvline(x=point[0], color="silver")

        xdose.sel(x=point[0], method="nearest").T.plot(ax=axes[0, 1], cmap="jet", vmin=0, vmax=xdose.max())
        axes[0, 1].axhline(y=point[2], color="silver")
        axes[0, 1].axvline(x=point[1], color="silver")

        xdose.sel(z=point[2], method="nearest").plot(ax=axes[1, 0], cmap="jet", vmin=0, vmax=xdose.max())
        axes[1, 0].axhline(y=point[1], color="silver")
        axes[1, 0].axvline(x=point[0], color="silver")
    except ValueError:
        pass

    fig.tight_layout()

In [ ]:
# Plot at point max
argmax = xdoses_by_orient["HFS"].argmax(...)

point_max = (
    xdoses_by_orient["HFS"].x[argmax['x']],
    xdoses_by_orient["HFS"].y[argmax['y']],
    xdoses_by_orient["HFS"].z[argmax['z']]
)

plot_xdose_tcs_at_point(xdose=xdoses_by_orient["HFS"], point=point_max, coord_system="SUPPORT")

In [ ]:
# Plot HFS cup at 

plot_xdose_tcs_at_point(xdose=xdoses_by_orient["HFS"], point=(0, -1157, 0), coord_system="SUPPORT")

In [ ]:
# plot HFP cup at centre

plot_xdose_tcs_at_point(xdose=xdoses_by_orient["HFP"], point=(0, -1157, 0), coord_system="SUPPORT")


In [ ]:
# plot FFP cup at centre

plot_xdose_tcs_at_point(xdose=xdoses_by_orient["FFP"], point=(0, -1157, 0), coord_system="SUPPORT")


In [ ]:
# plot FFDL cup at centre

plot_xdose_tcs_at_point(xdose=xdoses_by_orient["FFDL"], point=(0, -1157, 0), coord_system="SUPPORT")

In [ ]:
# zoom! Doesn't work yet :(
x_start = -55
x_end = 55
y_start = -1225
y_end = -1100
z_start = -50
z_end = 30
xdose_ffdl_zoomed = xdoses_by_orient["FFDL"].sel(x=slice(x_start, x_end), y=slice(y_start, y_end), z=slice(z_start, z_end))

plot_xdose_tcs_at_point(xdose=xdose_ffdl_zoomed, point=(0, -1157, 0), coord_system="SUPPORT")

In [ ]:
# Lovely inbuilt interpolation too!

x_new = np.linspace(xdoses_by_orient["FFDL"].x[0], xdoses_by_orient["FFDL"].x[-1], xdoses_by_orient["FFDL"].sizes["x"]*4) # Make 4x finer
y_new = np.linspace(xdoses_by_orient["FFDL"].y[0], xdoses_by_orient["FFDL"].y[-1], xdoses_by_orient["FFDL"].sizes["y"]*4) # Make 4x finer
z_new = np.linspace(xdoses_by_orient["FFDL"].z[0], xdoses_by_orient["FFDL"].z[-1], xdoses_by_orient["FFDL"].sizes["z"]*4) # Make 4x finer

xdose_ffdl_interp = xdoses_by_orient["FFDL"].interp(x=x_new, y=y_new, z=z_new)
xdose_ffdl_interp = xdoses_by_orient["FFDL"].sel(y=0, method="nearest")

In [ ]:
plot_xdose_tcs_at_point(xdose=xdose_ffdl_interp, point=(0, -1157, 0), coord_system="SUPPORT")